In [1]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from accelerate import Accelerator
from diffusers import UNet2DModel, DDPMScheduler, DDPMPipeline
from diffusers.optimization import get_scheduler
from torchvision.utils import save_image
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, Compose, CenterCrop

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'


In [2]:
transform = Compose([
    ToTensor(),
    CenterCrop([128, 128])
])
dataset = ImageFolder("./data/pokemini", transform=transform)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True, num_workers=0)

In [3]:
model = UNet2DModel(
sample_size=128,
in_channels=3,
out_channels=3,
layers_per_block=2,
block_out_channels=(128, 128, 256, 256, 512, 512),
down_block_types=(
"DownBlock2D", "DownBlock2D", "DownBlock2D", "DownBlock2D",
"AttnDownBlock2D",  "DownBlock2D",
),
up_block_types=(
"UpBlock2D", "AttnUpBlock2D",
"UpBlock2D", "UpBlock2D", "UpBlock2D", "UpBlock2D",
),
)

In [4]:
accelerator = Accelerator(
    gradient_accumulation_steps=1,
    mixed_precision="no"
)

In [5]:
noise_scheduler = DDPMScheduler(
    num_train_timesteps=1000
)

In [6]:
optimizer = torch.optim.AdamW(
    model.parameters(), lr=1E-4,
    betas=(0.95, 0.99),
    weight_decay=1E-6,
    eps=1E-8
)

In [7]:
lr_scheduler = get_scheduler(
    "cosine", optimizer=optimizer,
    num_warmup_steps=500,
    num_training_steps=(
        len(dataloader) * 100
    ),
)

In [8]:
model, optimizer, dataloader, lr_scheduler = accelerator.prepare(
    model, optimizer, dataloader, lr_scheduler
)

In [9]:
for epoch in range(100):
    model.train()
    for step, batch in enumerate(dataloader):
        clean_images = batch[0].to("cuda")
        noise = torch.randn(clean_images.shape).to(clean_images.device)
        timesteps = torch.randint(
            0, noise_scheduler.config.num_train_timesteps,
            (clean_images.shape[0],), device=clean_images.device
        ).long()
        
        # Add noise to the clean images according to the noise magnitude
        # at each timestep (this is the forward diffusion process)
        noisy_images = noise_scheduler.add_noise(
            clean_images, noise, timesteps
        )

        with accelerator.accumulate(model):
            # Predict the noise residual
            noise_pred = model(noisy_images, timesteps).sample
            loss = F.mse_loss(noise_pred, noise)
            accelerator.backward(loss)

            if accelerator.sync_gradients:
                accelerator.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
    print(f"Epoch {epoch+1} / 100 - Loss: {loss.detach().item()}")

    # Using  Will stop the execution of the current process
    # until every other process has reached that point.
    # This does nothing when running on a single process.
    accelerator.wait_for_everyone()

    
    # Generate sample images for visual inspection
    if accelerator.is_main_process:
        if epoch % 10 == 0 or epoch == 100 - 1:
            pipeline = DDPMPipeline(
                unet=accelerator.unwrap_model(model),
                scheduler=noise_scheduler,
            )

            generator = torch.manual_seed(0)
            # run pipeline in inference (sample random noise and denoise)
            images = pipeline(
                generator=generator,
                batch_size=64,
                output_type="numpy"
            ).images

            # denormalize the images and save to wandb
            processed_images = torch.tensor(images).transpose(1, 3).transpose(2, 3)
            save_image(processed_images, f"./hf_epoch-{epoch+1}.png")
            pipeline.save_pretrained("hf_poke.pth")

    accelerator.wait_for_everyone()

accelerator.end_training()

c:\Users\kril\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\Image.py:979: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1 / 100 - Loss: 0.18759840726852417


  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 2 / 100 - Loss: 0.205866739153862
Epoch 3 / 100 - Loss: 0.039883311837911606
Epoch 4 / 100 - Loss: 0.011146686971187592
Epoch 5 / 100 - Loss: 0.029633190482854843
Epoch 6 / 100 - Loss: 0.014035075902938843
Epoch 7 / 100 - Loss: 0.0035448407288640738
Epoch 8 / 100 - Loss: 0.004128352738916874
Epoch 9 / 100 - Loss: 0.0023402124643325806
Epoch 10 / 100 - Loss: 0.008877461776137352
Epoch 11 / 100 - Loss: 0.008535260334610939


  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 82 / 100 - Loss: 0.00900292582809925
Epoch 83 / 100 - Loss: 0.0054556829854846
Epoch 84 / 100 - Loss: 0.0007940351497381926
Epoch 85 / 100 - Loss: 0.006530681159347296
Epoch 86 / 100 - Loss: 0.0047417595051229
Epoch 87 / 100 - Loss: 0.0013857567682862282
Epoch 88 / 100 - Loss: 0.001885767444036901
Epoch 89 / 100 - Loss: 0.0005598432617262006
Epoch 90 / 100 - Loss: 0.008582176640629768
Epoch 91 / 100 - Loss: 0.005008901469409466


  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 92 / 100 - Loss: 0.018885962665081024
Epoch 93 / 100 - Loss: 0.008572029881179333
Epoch 94 / 100 - Loss: 0.0008478917879983783
Epoch 95 / 100 - Loss: 0.008590107783675194
Epoch 96 / 100 - Loss: 0.005641000345349312
Epoch 97 / 100 - Loss: 0.0006619002670049667
Epoch 98 / 100 - Loss: 0.004685807507485151
Epoch 99 / 100 - Loss: 0.00048288164543919265
Epoch 100 / 100 - Loss: 0.0025606052950024605


  0%|          | 0/1000 [00:00<?, ?it/s]